In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import json

Loading the empatica data

In [ ]:
dir_empatica = "<path to the data recorded in the empatica in Emognition dataset>"

elst = os.listdir(dir_empatica)

#find all unique ID of participants
temp = list()
for i in elst:
    temp.append(i.split("_")[0])
pid = list(set(temp))
print(pid)
print(len(pid))

#find all unique emmotions
temp2 = list()
for i in elst:
    temp2.append(i.split("_")[1])
emotion = list(set(temp2))
print(emotion)
print(len(emotion))

The Arousal Valence Mapping on the labels of the original dataset

In [3]:
arousal = {'SADNESS' :0,
           'DISGUST':0, 
           'AMUSEMENT' : 1, 
           'BASELINE' : 0, 
           'SURPRISE' : 1, 
           'LIKING' : 0, 
           'ANGER' : 1, 
           'NEUTRAL' : 0, 
           'ENTHUSIASM' : 1, 
           'FEAR' : 1, 
           'AWE' : 1}
valence = {'SADNESS' :0,
           'DISGUST':0, 
           'AMUSEMENT' : 1, 
           'BASELINE' : 0, 
           'SURPRISE' : 0, 
           'LIKING' : 1, 
           'ANGER' : 0, 
           'NEUTRAL' : 0, 
           'ENTHUSIASM' : 1, 
           'FEAR' : 0, 
           'AWE' : 1}

In [ ]:
path = f'{dir_empatica}/{elst}'
path

In [ ]:
dict_em_list = list() #All participant data in a dictionary

for i in range(len(elst)):
     # Open the JSON file
    print(i)
    with open(f"{dir_empatica}/{elst[i]}", 'r') as json_file:
        # Load the JSON data
        print(f"{dir_empatica}/{elst[i]}")
        json_data = json.load(json_file)
        dict_em_list.append(json_data)
        # break
    for key, value in json_data.items():
        print(f"Key: {key}")

In [6]:
empatica_files = os.listdir(dir_empatica)

Statistical Features

In [9]:
# cvxEDA
def eda_stats(y):
    Fs = 15.625
    yn = (y - y.mean()) / y.std()
    [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(yn, 1. / Fs)
    return [r, p, t, l, d, e, obj]

def shannon_entropy(window):
    p = np.abs(window) / np.sum(np.abs(window))
    return -np.sum(p * np.log2(p + 1e-10))

def first_derivative(signal):
    if len(signal) > 1:
        time_values = np.arange(len(signal))
        first_derivative = np.gradient(signal, time_values)
        return first_derivative
    else:
        return np.array([])


def second_derivative(signal):
    fd = first_derivative(signal)
    time_values = np.arange(len(fd))
    second_derivative = np.gradient(first_derivative)
    return second_derivative


def calculate_integral(window):
    a = np.sum(np.abs(window))
    return a

def calculate_avg_power(window):
    avg_power = np.mean(np.square(np.abs(window)))
    return avg_power

def calculate_arc_length(window):
    diff_signal = np.diff(window)
    arc_length = np.sum(np.sqrt(1 + np.square(diff_signal)))
    return arc_length

def slope(window):
    if len(window) > 1:
        time_values = np.arange(len(window))
        slope, _ = np.polyfit(time_values, window, 1)
        return slope
    else:
        return np.nan

In [7]:
import numpy as np
import pandas as pd
import os
import neurokit2 as nk
import cvxEDA.src.cvxEDA as cvxEDA
import scipy.stats as stats
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
sampling_acc = 32

Feature Extraction

In [ ]:
out_df = pd.DataFrame()
for fle in empatica_files:
    fte = list(fle.split("_"))
    pid, arousal_l, valence_l, setting, emotion = fte[0], arousal[fte[1]], valence[fte[1]], fte[2], fte[1]
    data = json.load(open(dir_empatica + "/" + fle))
    sampling_eda = 4
    eda_data = []
    for i in data['EDA']:
        eda_data.append(i[1])

    print(len(eda_data))
    eda_clean = nk.eda_clean(eda_data, sampling_rate=sampling_eda ) #Cleaning eda signal using NK
    x = np.array(eda_clean)
    eda = nk.eda_phasic(x, sampling_eda)
    scr = np.array(eda['EDA_Phasic'])
    scl = np.array(eda['EDA_Tonic'])
    x_axis = np.linspace(0, scl.shape[0]/sampling_eda, scl.shape[0])
    row_dict = {}
    
    #raw eda features
    row_dict['mean'] = np.mean(x) # Mean
    row_dict['std'] = np.std(x) # Standard Deviation
    row_dict['min'] = np.min(x) # Minimum
    row_dict['max'] = np.max(x) # Maximum
    row_dict['median_eda'] = np.quantile(x,0.5) #median
    row_dict['ku_eda'] = stats.kurtosis(x) #kurtosis
    row_dict['sk_eda'] = stats.skew(x) #skewness
    row_dict['dynrange'] = x.max()/x.min()#dynamic range
    row_dict['slope'] = np.polyfit(x_axis,scl,1)[0] #slope
    row_dict['variance'] = np.var(x) # Variance
    row_dict['entropy'] = shannon_entropy(x) # Shannon Entropy
    row_dict['insc'] = calculate_integral(x) # insc
    fd = first_derivative(x)
    row_dict['fd_mean'] = np.mean(fd)
    row_dict['fd_std'] = np.std(fd)

    #scr features
    row_dict['max_scr'] = np.max(scr) #min
    row_dict['min_scr'] = np.min(scr) #max
    row_dict['mean_scr'] = np.mean(scr) # Mean
    row_dict['sd_scr'] = np.std(scr) # Standard Deviation

    _, info = nk.eda_peaks(scr, sampling_eda) #scr peak
    peaks = info['SCR_Peaks']
    amplitude = info['SCR_Amplitude']
    recovery = info['SCR_RecoveryTime']
    
    row_dict['nSCR'] = len(info['SCR_Peaks']) / (x.shape[0]/sampling_eda/60) #to get the number of peaks per minute
    row_dict['aucSCR'] = np.trapz(scr)
    row_dict['meanAmpSCR'] = np.nanmean(amplitude)
    row_dict['maxAmpSCR'] = np.nanmax(amplitude)
    row_dict['meanRespSCR'] = np.nanmean(recovery)
    row_dict['sumAmpSCR'] = np.nansum(amplitude) / (x.shape[0]/sampling_eda/60) # per minute
    row_dict['sumRespSCR'] = np.nansum(recovery) / (x.shape[0]/sampling_eda/60) # per minute

    #scl features
    row_dict['max_scl'] = np.max(scl) #min
    row_dict['min_scl'] = np.min(scl) #max
    row_dict['mean_scl'] = np.mean(scl) # Mean
    row_dict['sd_scl'] = np.std(scl) # Standard Deviation

    row_dict['Participant ID'] = pid
    row_dict['arousal_category'] = arousal_l
    row_dict['valence_category'] = valence_l
    row_dict['Setting'] = setting
    row_dict['Emotion'] = emotion
    print(row_dict.keys())

    new_row = pd.DataFrame(row_dict , index=[0])
    out_df = pd.concat([out_df, new_row], ignore_index=True)


out_df    

In [ ]:
out_df.keys()

In [ ]:
columns_with_nan = out_df.columns[out_df.isna().any()].tolist()
columns_with_nan

In [11]:
relevant_features = ['ku_eda','sk_eda','dynrange','slope','variance','entropy','insc','fd_mean',
                     'max_scr','min_scr','nSCR','meanAmpSCR','meanRespSCR','sumAmpSCR','sumRespSCR']


identifiers = ['Participant ID','arousal_category', 'valence_category', 'Setting', 'Emotion']

In [ ]:
a_list = out_df['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = out_df['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

In [ ]:
set(out_df["Setting"].tolist())

In [ ]:
set(out_df["Emotion"].tolist())

In [ ]:
len(out_df)

In [ ]:
out_df1 = out_df[out_df["Setting"].isin(['QUESTIONNAIRES', 'STIMULUS'])]
eda_fet = out_df1[['mean', 'std', 'min', 'max', 'median_eda', 'ku_eda', 'sk_eda',
       'dynrange', 'slope', 'variance', 'entropy', 'insc', 'fd_mean', 'fd_std',
       'max_scr', 'min_scr', 'mean_scr', 'sd_scr', 'nSCR', 'aucSCR',
       'meanAmpSCR', 'maxAmpSCR', 'meanRespSCR', 'sumAmpSCR', 'sumRespSCR',
       'max_scl', 'min_scl', 'mean_scl', 'sd_scl', 'Participant ID','arousal_category', 'valence_category', 'Setting', 'Emotion']]

eda_fet

In [ ]:
scaler = MinMaxScaler()

# List of columns to be scaled
columns_to_scale = eda_fet.columns.difference(['Participant ID', 'arousal_category', 'valence_category', 'Setting', 'Emotion'])

# Function to scale participant-wise
def scale_participant(df):
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    return df

# Apply the scaling function to each participant's data
scaled_eda_fet = eda_fet.groupby('Participant ID').apply(scale_participant).reset_index(drop=True)

scaled_eda_fet

In [ ]:
a_list = scaled_eda_fet['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = scaled_eda_fet['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

In [ ]:
set(scaled_eda_fet['Setting'].tolist())

In [ ]:
stimulus_eda = scaled_eda_fet[scaled_eda_fet['Setting'] == 'STIMULUS'].reset_index(drop=True)
a_list = stimulus_eda['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = stimulus_eda['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)
stimulus_eda

In [ ]:
pers_eda = scaled_eda_fet[scaled_eda_fet['Setting'] == 'QUESTIONNAIRES'].reset_index(drop=True)
a_list = pers_eda['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = pers_eda['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)
pers_eda

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as mp
import seaborn as sb
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from sklearn.metrics import classification_report
from torchmetrics import F1Score
from tqdm.autonotebook import tqdm
import itertools
import random 
import os

Code for checking zero shot transferability

In [24]:
def csv_read(path):
    df = pd.read_csv(path)
    return df

def lowercase(data):
  temp  = data.lower()
  return temp

def stop_words(text):
  stop_words_set = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words_set]
  return " ".join(filtered_sentence)

def punctuations(data):
  no_punct=[words for words in data if words not in string.punctuation]
  words_wo_punct=''.join(no_punct)
  return words_wo_punct

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  word_tokens = word_tokenize(text)
  lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
  return " ".join(lemmatized_text)

def preprcosess(text_data):
    text_data['Text'] = text_data['Text Description'].apply(lambda x: lemmatize(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: stop_words(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: lowercase(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: punctuations(x))
    return text_data

def balanced_df(eda_df):
    filtered_rows = []
    for index, row in tqdm(eda_df.iterrows()):
        if row['arousal_category'] == 1:
            filtered_rows.append(row)

    filtered_df = pd.DataFrame(filtered_rows)
    balance_a_eda_df = pd.concat([eda_df, filtered_df], ignore_index=True)
    balance_a_eda_df  = balance_a_eda_df.reset_index(drop=True)
    return balance_a_eda_df

class Text_Encoder(nn.Module):
    def __init__(self, model_name="distilbert-base-uncased", pretrained=True, trainable=True):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name).to(torch.device("cuda"))
            
        for p in self.model.parameters():
            p.requires_grad = trainable
        
        for p in self.model.parameters():
            p.requires_grad = trainable
        


    def text_tokens(self,batch):
        text_embeddings = []
        for i in range(len(batch)):
            texts = batch[i]
            tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
            model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(torch.device("cuda"))
                
            # Tokenize and get embeddings
            encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(torch.device("cuda"))
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Extract embeddings from the last hidden state
            embeddings = model_output.last_hidden_state

            # Get the embeddings for the [CLS] token (the first token)
            cls_embeddings = embeddings[:, 0, :]

            # Alternatively, mean pooling the token embeddings to get sentence-level embeddings
            sentence_embeddings = torch.mean(embeddings, dim=1)

            text_embeddings.append(sentence_embeddings)

        return text_embeddings
    
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=100,  # Change projection_dim to 100
        dropout=0.1
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, batch):
        text_embeddings = []
        for i in range(len(batch)):
            x = batch[i]
            projected = self.projection(x)
            x = self.gelu(projected)
            x = self.fc(x)
            x = self.dropout(x)
            x = x + projected
            x = self.layer_norm(x)
            text_embeddings.append(x)
        return text_embeddings
    
class CustomMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomMLP, self).__init__()

        # Define hidden layer dimensions
        hidden_dims = [50, 100]

        # Create sequential layers using nn.Linear and nn.ReLU activations
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(inplace=True)
        )
        self.output_layer = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.output_layer(x)
        return x

    def get_hidden_embedding(self, x):
        x = self.layer1(x)
        return self.layer2(x)
    
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, text):
        self.data = data
        self.targets = targets
        self.text  = text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = {}
        item['data'] = torch.from_numpy(self.data[index]).float()
        item['target'] = self.targets[index]
        item['text'] = self.text[index]
        return item
    
def windowed_preprocess(train_df, test_df):
    train = []
    test = []
    for index, row in train_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list1 = np.array([np.array(value) for value in row.to_numpy()])
        train.append(row_as_list1)

    for index, row in test_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list2 = np.array([np.array(value) for value in row.to_numpy()])
        test.append(row_as_list2)

    return train, test

class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

class CLIPModel(nn.Module):
    def __init__(self, mlp_input_dim, mlp_output_dim, device):
        super().__init__()
        self.text_encoder = Text_Encoder().to(device)
        self.eda_encoder = CustomMLP(mlp_input_dim, mlp_output_dim).to(device)
        self.text_projection = ProjectionHead(embedding_dim=768, projection_dim=100).to(device)
        self.device = device
        
    def eda_train(self,learning_rate, beta1, beta2 , epsilon , train_dataloader):
        criterion= nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam( self.eda_encoder.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=epsilon)
        optimizer.zero_grad()
        for batch in tqdm(train_dataloader):
            data = batch['data']
            target = batch['target']

            data, target = data.to(self.device), target.to(self.device)  # Move data and target to GPU
            optimizer.zero_grad()

            output = self.eda_encoder(data)

            target = target.unsqueeze(1).float()
            
            # Calculate loss
            loss = criterion(output, target)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

    def forward(self, batch):
        data_values = batch['data']
        text_values = batch['text']

        eda_embeddings = self.eda_encoder.get_hidden_embedding(data_values).to(self.device)
        text_embeddings  = self.text_encoder.text_tokens(batch["text"])#.to(self.device)
        text_embeddings = self.text_projection(text_embeddings)#.to(self.device)
        text_embeddings = torch.stack(text_embeddings).to(self.device)
        eda_tensor = eda_embeddings

        # Calculating the Loss
        text_embeddings = text_embeddings.squeeze(1)
        logits = torch.matmul(text_embeddings, eda_tensor.T)
        # print("Logits ",logits.shape)
        eda_similarity = torch.matmul(eda_tensor, eda_tensor.T)
        # print("eda_similarity ",eda_similarity.shape)
        input_size = text_embeddings.size(0) * text_embeddings.size(1)
        # print("Text Embedding ",text_embeddings.shape)
        texts_similarity = torch.matmul(text_embeddings, text_embeddings.T)
        targets = F.softmax((eda_similarity + texts_similarity) / 2, dim=-1) 
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

def train_epoch(model, train_loader, optimizer, lr_scheduler, step, device=torch.device("cuda")):
    # device = torch.device("cuda")
    model.to(device)
    loss_meter = AvgMeter()
    # print(f"train_loader: {train_loader}")
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        for key in batch.keys():
            if key != "text":
                # print(key)
                batch[key] = batch[key].to(device)
            
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()
        count = batch["data"].size(0)
        loss_meter.update(loss.item(), count)
        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter

In [25]:
input_dim =len(relevant_features) # EDA Features Dimension
output_dim = 1
device = torch.device("cuda")

In [ ]:
fet = relevant_features
for i in range(1,41):
    label_id = 2
    model_path = "<Path to the pre trained model>"
    X_test = stimulus_eda[fet]
    test_y = stimulus_eda['arousal_category'].to_list()
    label1 = "Data of High and more arousal_category"
    label2 = "Data of Low and less arousal_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)

        for j in X_test:
            if isinstance(j, np.ndarray):
                features = best_model.eda_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
                features.to(device)
                a = torch.matmul(features, encoded_label1.T) 
                b = torch.matmul(features, encoded_label2.T)
                # print(a)
                # print(b)
                value = 1 if a > b else 0
                pred.append(value)
            else:
                print(f"Skipping invalid entry: {j}")
            
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)
        print(f"Label Arousal, {i}")
        print(f"accuracy {accuracy}")
        print(f"f1 {f1}")

    except Exception as e:
        print(e)


In [ ]:
fet = relevant_features
for i in range(1,41):
    label_id = 1
    model_path = "<Path to the pre trained model>"
    X_test = stimulus_eda[fet]
    test_y = stimulus_eda['valence_category'].to_list()
    label1 = "Data of High and more valence_category"
    label2 = "Data of Low and less valence_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)

        for j in X_test:
            if isinstance(j, np.ndarray):
                features = best_model.eda_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
                features.to(device)
                a = torch.matmul(features, encoded_label1.T) 
                b = torch.matmul(features, encoded_label2.T)
                # print(a)
                # print(b)
                value = 1 if a > b else 0
                pred.append(value)
            else:
                print(f"Skipping invalid entry: {j}")
            
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)
        print(f"Label Valence, {i}")
        print(f"accuracy {accuracy}")
        print(f"f1 {f1}")

    except Exception as e:
        print(e)


PPG

In [ ]:
out_ppg = pd.DataFrame()
for fle in empatica_files:
    fte = list(fle.split("_"))
    pid, arousal_l, valence_l, setting, emotion = fte[0], arousal[fte[1]], valence[fte[1]], fte[2], fte[1]
    data = json.load(open(dir_empatica + "/" + fle))
    sampling_eda = 4
    ppgdata = []
    for i in data['BVP']:
        ppgdata.append(i[1])

    ibidata =[]
    for i in data['IBI']:
        ibidata.append(i[1])


    sampling_acc = 32

    row_dict = {}
    row_dict['Participant ID'] = pid
    row_dict['arousal_category'] = arousal_l
    row_dict['valence_category'] = valence_l
    row_dict['Setting'] = setting
    row_dict['Emotion'] = emotion

    try:
        bvp_clean = nk.ppg_clean(ppgdata, sampling_rate=sampling_acc)
            
        # Perform BVP analysis if the signal is long enough
        if len(bvp_clean) > 10 * sampling_acc:  # Ensure at least 10 seconds of data
            bvp_signals, bvp_info = nk.ppg_process(bvp_clean, sampling_rate=sampling_acc)
            
            # Perform the analysis
            analyze_df = nk.ppg_analyze(bvp_signals, sampling_rate=sampling_acc)
            
            # Update your data with analysis results
            row_dict.update(analyze_df.iloc[0])
            heart_rate = bvp_signals['PPG_Rate']
            bpm = heart_rate.mean()
            print(bpm)
            row_dict['BPM'] = bpm
            if len(ibidata) == 0:
                row_dict['IBI'] = 0
            else:
                row_dict['IBI'] = sum(ibidata)/len(ibidata)
            new_row = pd.DataFrame(row_dict , index=[0])
            out_ppg = pd.concat([out_ppg, new_row], ignore_index=True)

        else:
            print(f"Signal too short for analysis at index")
            continue

    except Exception as e:
        print(f"Error processing signal at index: {e}")
        continue

    print(row_dict.keys())

    

out_ppg


In [ ]:
out_ppg.keys()

In [ ]:
columns_with_nan = out_ppg.columns[out_ppg.isna().any()].tolist()
columns_with_nan

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as mp
import seaborn as sb
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from sklearn.metrics import classification_report
from torchmetrics import F1Score
from tqdm.autonotebook import tqdm
import itertools
import random 
import os


warnings.filterwarnings('ignore')

def csv_read(path):
    df = pd.read_csv(path)
    return df

def lowercase(data):
  temp  = data.lower()
  return temp

def stop_words(text):
  stop_words_set = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words_set]
  return " ".join(filtered_sentence)

def punctuations(data):
  no_punct=[words for words in data if words not in string.punctuation]
  words_wo_punct=''.join(no_punct)
  return words_wo_punct

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  word_tokens = word_tokenize(text)
  lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
  return " ".join(lemmatized_text)

def preprcosess(text_data):
    text_data['Text'] = text_data['Text Description'].apply(lambda x: lemmatize(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: stop_words(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: lowercase(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: punctuations(x))
    return text_data

def balanced_df(df):
    filtered_rows = []
    for index, row in tqdm(df.iterrows()):
        if row['arousal_category'] == 1:
            filtered_rows.append(row)

    filtered_df = pd.DataFrame(filtered_rows)
    balance_a_df = pd.concat([df, filtered_df], ignore_index=True)
    balance_a_df  = balance_a_df.reset_index(drop=True)
    return balance_a_df

class Text_Encoder(nn.Module):
    def __init__(self, model_name="distilbert-base-uncased", pretrained=True, trainable=True):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name).to(torch.device("cuda"))
            
        for p in self.model.parameters():
            p.requires_grad = trainable
        
        for p in self.model.parameters():
            p.requires_grad = trainable
        


    def text_tokens(self,batch):
        text_embeddings = []
        for i in range(len(batch)):
            texts = batch[i]
            tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
            model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(torch.device("cuda"))
                
            # Tokenize and get embeddings
            encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(torch.device("cuda"))
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Extract embeddings from the last hidden state
            embeddings = model_output.last_hidden_state

            # Get the embeddings for the [CLS] token (the first token)
            cls_embeddings = embeddings[:, 0, :]

            # Alternatively, mean pooling the token embeddings to get sentence-level embeddings
            sentence_embeddings = torch.mean(embeddings, dim=1)

            text_embeddings.append(sentence_embeddings)

        return text_embeddings
    
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=100,  # Change projection_dim to 100
        dropout=0.1
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, batch):
        text_embeddings = []
        for i in range(len(batch)):
            x = batch[i]
            projected = self.projection(x)
            x = self.gelu(projected)
            x = self.fc(x)
            x = self.dropout(x)
            x = x + projected
            x = self.layer_norm(x)
            text_embeddings.append(x)
        return text_embeddings
    
class CustomMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomMLP, self).__init__()

        # Define hidden layer dimensions
        hidden_dims = [50, 100]

        # Create sequential layers using nn.Linear and nn.ReLU activations
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(inplace=True)
        )
        self.output_layer = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.output_layer(x)
        return x

    def get_hidden_embedding(self, x):
        x = self.layer1(x)
        return self.layer2(x)
    
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, text):
        self.data = data
        self.targets = targets
        self.text  = text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = {}
        item['data'] = torch.from_numpy(self.data[index]).float()
        item['target'] = self.targets[index]
        item['text'] = self.text[index]
        return item
    
def windowed_preprocess(train_df, test_df):
    train = []
    test = []
    for index, row in train_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list1 = np.array([np.array(value) for value in row.to_numpy()])
        train.append(row_as_list1)

    for index, row in test_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list2 = np.array([np.array(value) for value in row.to_numpy()])
        test.append(row_as_list2)

    return train, test

class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

class CLIPModel(nn.Module):
    def __init__(self, mlp_input_dim, mlp_output_dim, device):
        super().__init__()
        self.text_encoder = Text_Encoder().to(device)
        self.ppg_encoder = CustomMLP(mlp_input_dim, mlp_output_dim).to(device)
        self.text_projection = ProjectionHead(embedding_dim=768, projection_dim=100).to(device)
        self.device = device
        
    def ppg_train(self,learning_rate, beta1, beta2 , epsilon , train_dataloader):
        criterion= nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam( self.ppg_encoder.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=epsilon)
        optimizer.zero_grad()
        for batch in tqdm(train_dataloader):
            data = batch['data']
            target = batch['target']

            data, target = data.to(self.device), target.to(self.device)  # Move data and target to GPU
            optimizer.zero_grad()

            output = self.ppg_encoder(data)

            target = target.unsqueeze(1).float()
            
            # Calculate loss
            loss = criterion(output, target)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

    def forward(self, batch):
        data_values = batch['data']
        text_values = batch['text']

        ppg_embeddings = self.ppg_encoder.get_hidden_embedding(data_values).to(self.device)
        text_embeddings  = self.text_encoder.text_tokens(batch["text"])#.to(self.device)
        text_embeddings = self.text_projection(text_embeddings)#.to(self.device)
        text_embeddings = torch.stack(text_embeddings).to(self.device)
        ppg_tensor = ppg_embeddings

        # Calculating the Loss
        text_embeddings = text_embeddings.squeeze(1)
        logits = torch.matmul(text_embeddings, ppg_tensor.T)
        # print("Logits ",logits.shape)
        ppg_similarity = torch.matmul(ppg_tensor, ppg_tensor.T)
        input_size = text_embeddings.size(0) * text_embeddings.size(1)
        # print("Text Embedding ",text_embeddings.shape)
        texts_similarity = torch.matmul(text_embeddings, text_embeddings.T)
        targets = F.softmax((ppg_similarity + texts_similarity) / 2, dim=-1) 
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

def train_epoch(model, train_loader, optimizer, lr_scheduler, step, device=torch.device("cuda")):
    # device = torch.device("cuda")
    model.to(device)
    loss_meter = AvgMeter()
    # print(f"train_loader: {train_loader}")
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        for key in batch.keys():
            if key != "text":
                # print(key)
                batch[key] = batch[key].to(device)
            
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()
        count = batch["data"].size(0)
        loss_meter.update(loss.item(), count)
        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter

In [16]:
relevant_features_ppg = ['BPM', 'IBI', 'PPG_Rate_Mean', 'HRV_MedianNN',
       'HRV_Prc20NN', 'HRV_MinNN', 'HRV_HTI', 'HRV_TINN', 'HRV_LF', 'HRV_VHF',
       'HRV_LFn', 'HRV_HFn', 'HRV_LnHF', 'HRV_SD1SD2', 'HRV_CVI', 'HRV_PSS',
       'HRV_PAS', 'HRV_PI', 'HRV_C1d', 'HRV_C1a', 'HRV_DFA_alpha1',
       'HRV_MFDFA_alpha1_Width', 'HRV_MFDFA_alpha1_Peak',
       'HRV_MFDFA_alpha1_Mean', 'HRV_MFDFA_alpha1_Max',
       'HRV_MFDFA_alpha1_Delta', 'HRV_MFDFA_alpha1_Asymmetry', 'HRV_ApEn',
       'HRV_ShanEn', 'HRV_FuzzyEn', 'HRV_MSEn', 'HRV_CMSEn', 'HRV_RCMSEn',
       'HRV_CD', 'HRV_HFD', 'HRV_KFD', 'HRV_LZC']

identifiers = ['Participant ID','arousal_category', 'valence_category', 'Setting', 'Emotion']

In [ ]:
a_list = out_ppg['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = out_ppg['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

In [ ]:
set(out_df["Setting"].tolist())

In [ ]:
set(out_df["Emotion"].tolist())

In [ ]:
out_ppg1 = out_ppg[out_ppg["Setting"].isin(['QUESTIONNAIRES', 'STIMULUS'])]
ppg_fet = out_ppg1[relevant_features_ppg + identifiers]

ppg_fet

In [ ]:
scaler = MinMaxScaler()

# List of columns to be scaled
columns_to_scale = ppg_fet.columns.difference(['Participant ID', 'arousal_category', 'valence_category', 'Setting', 'Emotion'])

# Function to scale participant-wise
def scale_participant(df):
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    return df

# Apply the scaling function to each participant's data
scaled_ppg_fet = ppg_fet.groupby('Participant ID').apply(scale_participant).reset_index(drop=True)

scaled_ppg_fet

In [ ]:
a_list = scaled_ppg_fet['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = scaled_ppg_fet['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

In [ ]:
stimulus_ppg = scaled_ppg_fet[scaled_ppg_fet['Setting'] == 'STIMULUS'].reset_index(drop=True)
a_list = stimulus_ppg['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = stimulus_ppg['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)
stimulus_ppg

In [ ]:
pers_ppg = scaled_ppg_fet[scaled_ppg_fet['Setting'] == 'QUESTIONNAIRES'].reset_index(drop=True)
a_list = pers_ppg['arousal_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)

a_list = pers_ppg['valence_category'].tolist()
l = len(a_list)
o = sum(a_list)
z = l - o
print(o,z,l)
pers_ppg

In [46]:
input_dim =len(relevant_features_ppg) # EDA Features Dimension
output_dim = 1
device = torch.device("cuda")

In [ ]:
fet = relevant_features_ppg
for i in range(1,41):
    label_id = 2
    model_path = "<Path to the pretrained model>"
    X_test = stimulus_ppg[fet]
    test_y = stimulus_ppg['arousal_category'].to_list()
    label1 = "Data of High and more arousal_category"
    label2 = "Data of Low and less arousal_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)
        for j in X_test:
            features = best_model.ppg_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
            # print(features.shape)
            # print(encoded_label1.shape)
            # print(encoded_label2.shape)
            a = torch.matmul(features, encoded_label1.T) 
            b = torch.matmul(features, encoded_label2.T)
            # print(a)
            # print(b)
            value = 1 if a > b else 0
            pred.append(value)

        print(i,"arousal")
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)

        print(f"Accuracy {accuracy}")
        print(f"Fi score {f1}")

    except Exception as e:
        print(e)

In [ ]:
fet = relevant_features_ppg
for i in range(1,41):
    label_id = 1
    model_path = "<Path to the pre trained model>"
    X_test = stimulus_ppg[fet]
    test_y = stimulus_ppg['valence_category'].to_list()
    label1 = "Data of High and more valence_category"
    label2 = "Data of Low and less valence_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)
        for j in X_test:
            features = best_model.ppg_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
            # print(features.shape)
            # print(encoded_label1.shape)
            # print(encoded_label2.shape)
            a = torch.matmul(features, encoded_label1.T) 
            b = torch.matmul(features, encoded_label2.T)
            # print(a)
            # print(b)
            value = 1 if a > b else 0
            pred.append(value)

        print(i, "valence")
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)

        print(f"Accuracy {accuracy}")
        print(f"Fi score {f1}")

    except Exception as e:
        print(e)

EDA + PPG

In [20]:
scaled_combined =  pd.concat([scaled_eda_fet, scaled_ppg_fet], axis=1)
scaled_combined = scaled_combined.loc[:, ~scaled_combined.columns.duplicated()]
scaled_combined = scaled_combined.reset_index(drop=True)

# pers_combined =  pd.concat([pers_eda, pers_ppg], axis=1)
# pers_combined = pers_combined.loc[:, ~pers_combined.columns.duplicated()]
# pers_combined = pers_combined.reset_index(drop=True)

# stimulus_combined =  pd.concat([stimulus_eda, stimulus_ppg], axis=1)
# stimulus_combined = stimulus_combined.loc[:, ~stimulus_combined.columns.duplicated()]
# stimulus_combined = stimulus_combined.reset_index(drop=True)

relevant_features_combined = ['BPM', 'IBI', 'PPG_Rate_Mean', 'HRV_MedianNN', 'HRV_Prc20NN',
    'HRV_MinNN', 'HRV_HTI', 'HRV_TINN', 'HRV_LF', 'HRV_VHF', 'HRV_LFn',
    'HRV_HFn', 'HRV_LnHF', 'HRV_SD1SD2', 'HRV_CVI', 'HRV_PSS', 'HRV_PAS',
    'HRV_PI', 'HRV_C1d', 'HRV_C1a', 'HRV_DFA_alpha1',
    'HRV_MFDFA_alpha1_Width', 'HRV_MFDFA_alpha1_Peak',
    'HRV_MFDFA_alpha1_Mean', 'HRV_MFDFA_alpha1_Max',
    'HRV_MFDFA_alpha1_Delta', 'HRV_MFDFA_alpha1_Asymmetry', 'HRV_ApEn',
    'HRV_ShanEn', 'HRV_FuzzyEn', 'HRV_MSEn', 'HRV_CMSEn', 'HRV_RCMSEn',
    'HRV_CD', 'HRV_HFD', 'HRV_KFD', 'HRV_LZC', 'ku_eda', 'sk_eda', 'dynrange', 'slope',
    'variance', 'entropy', 'insc', 'fd_mean', 'max_scr', 'min_scr', 'nSCR',
    'meanAmpSCR', 'meanRespSCR', 'sumAmpSCR', 'sumRespSCR']

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as mp
import seaborn as sb
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from sklearn.metrics import classification_report
from torchmetrics import F1Score
from tqdm.autonotebook import tqdm
import itertools
import random 
import os


warnings.filterwarnings('ignore')

def csv_read(path):
    df = pd.read_csv(path)
    return df

def lowercase(data):
  temp  = data.lower()
  return temp

def stop_words(text):
  stop_words_set = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words_set]
  return " ".join(filtered_sentence)

def punctuations(data):
  no_punct=[words for words in data if words not in string.punctuation]
  words_wo_punct=''.join(no_punct)
  return words_wo_punct

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  word_tokens = word_tokenize(text)
  lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
  return " ".join(lemmatized_text)

def preprcosess(text_data):
    text_data['Text'] = text_data['Text Description'].apply(lambda x: lemmatize(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: stop_words(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: lowercase(x))
    text_data['Text'] = text_data['Text'].apply(lambda x: punctuations(x))
    return text_data

def balanced_df(df):
    filtered_rows = []
    for index, row in tqdm(df.iterrows()):
        if row['arousal_category'] == 1:
            filtered_rows.append(row)

    filtered_df = pd.DataFrame(filtered_rows)
    balance_a_df = pd.concat([df, filtered_df], ignore_index=True)
    balance_a_df  = balance_a_df.reset_index(drop=True)
    return balance_a_df

class Text_Encoder(nn.Module):
    def __init__(self, model_name="distilbert-base-uncased", pretrained=True, trainable=True):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name).to(torch.device("cuda"))
            
        for p in self.model.parameters():
            p.requires_grad = trainable
        
        for p in self.model.parameters():
            p.requires_grad = trainable
        


    def text_tokens(self,batch):
        text_embeddings = []
        for i in range(len(batch)):
            texts = batch[i]
            tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
            model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(torch.device("cuda"))
                
            # Tokenize and get embeddings
            encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(torch.device("cuda"))
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Extract embeddings from the last hidden state
            embeddings = model_output.last_hidden_state

            # Get the embeddings for the [CLS] token (the first token)
            cls_embeddings = embeddings[:, 0, :]

            # Alternatively, mean pooling the token embeddings to get sentence-level embeddings
            sentence_embeddings = torch.mean(embeddings, dim=1)

            text_embeddings.append(sentence_embeddings)

        return text_embeddings
    
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=100,  # Change projection_dim to 100
        dropout=0.1
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, batch):
        text_embeddings = []
        for i in range(len(batch)):
            x = batch[i]
            projected = self.projection(x)
            x = self.gelu(projected)
            x = self.fc(x)
            x = self.dropout(x)
            x = x + projected
            x = self.layer_norm(x)
            text_embeddings.append(x)
        return text_embeddings
    
class CustomMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomMLP, self).__init__()

        # Define hidden layer dimensions
        hidden_dims = [50, 100]

        # Create sequential layers using nn.Linear and nn.ReLU activations
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(inplace=True)
        )
        self.output_layer = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.output_layer(x)
        return x

    def get_hidden_embedding(self, x):
        x = self.layer1(x)
        return self.layer2(x)
    
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, text):
        self.data = data
        self.targets = targets
        self.text  = text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = {}
        item['data'] = torch.from_numpy(self.data[index]).float()
        item['target'] = self.targets[index]
        item['text'] = self.text[index]
        return item
    
def windowed_preprocess(train_df, test_df):
    train = []
    test = []
    for index, row in train_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list1 = np.array([np.array(value) for value in row.to_numpy()])
        train.append(row_as_list1)

    for index, row in test_df.iterrows():
        # Convert each row (Series) into a list of NumPy arrays
        row_as_list2 = np.array([np.array(value) for value in row.to_numpy()])
        test.append(row_as_list2)

    return train, test

class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

class CLIPModel(nn.Module):
    def __init__(self, mlp_input_dim, mlp_output_dim, device):
        super().__init__()
        self.text_encoder = Text_Encoder().to(device)
        self.combined_encoder = CustomMLP(mlp_input_dim, mlp_output_dim).to(device)
        self.text_projection = ProjectionHead(embedding_dim=768, projection_dim=100).to(device)
        self.device = device
        
    def combined_train(self,learning_rate, beta1, beta2 , epsilon , train_dataloader):
        criterion= nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam( self.combined_encoder.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=epsilon)
        optimizer.zero_grad()
        for batch in tqdm(train_dataloader):
            data = batch['data']
            target = batch['target']

            data, target = data.to(self.device), target.to(self.device)  # Move data and target to GPU
            optimizer.zero_grad()

            output = self.combined_encoder(data)

            target = target.unsqueeze(1).float()
            
            # Calculate loss
            loss = criterion(output, target)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

    def forward(self, batch):
        data_values = batch['data']
        text_values = batch['text']

        combined_embeddings = self.combined_encoder.get_hidden_embedding(data_values).to(self.device)
        text_embeddings  = self.text_encoder.text_tokens(batch["text"])#.to(self.device)
        text_embeddings = self.text_projection(text_embeddings)#.to(self.device)
        text_embeddings = torch.stack(text_embeddings).to(self.device)
        combined_tensor = combined_embeddings

        # Calculating the Loss
        text_embeddings = text_embeddings.squeeze(1)
        logits = torch.matmul(text_embeddings, combined_tensor.T)
        # print("Logits ",logits.shape)
        combined_similarity = torch.matmul(combined_tensor, combined_tensor.T)
        input_size = text_embeddings.size(0) * text_embeddings.size(1)
        # print("Text Embedding ",text_embeddings.shape)
        texts_similarity = torch.matmul(text_embeddings, text_embeddings.T)
        targets = F.softmax((combined_similarity + texts_similarity) / 2, dim=-1) 
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

def train_epoch(model, train_loader, optimizer, lr_scheduler, step, device=torch.device("cuda")):
    # device = torch.device("cuda")
    model.to(device)
    loss_meter = AvgMeter()
    # print(f"train_loader: {train_loader}")
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        for key in batch.keys():
            if key != "text":
                # print(key)
                batch[key] = batch[key].to(device)
            
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()
        count = batch["data"].size(0)
        loss_meter.update(loss.item(), count)
        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter

In [ ]:
input_dim = len(relevant_features_combined)
fet = relevant_features_combined
for i in range(1,41):
    label_id = 2
    model_path = "<Path to the pretrained model>"
    X_test = stimulus_combined[fet]
    test_y = stimulus_combined['arousal_category'].to_list()
    label1 = "Data of High and more arousal_category"
    label2 = "Data of Low and less arousal_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)
        for j in X_test:
            features = best_model.combined_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
            # print(features.shape)
            # print(encoded_label1.shape)
            # print(encoded_label2.shape)
            a = torch.matmul(features, encoded_label1.T) 
            b = torch.matmul(features, encoded_label2.T)
            # print(a)
            # print(b)
            value = 1 if a > b else 0
            pred.append(value)

        print(i, "arousal")
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)

        print(f"Accuracy {accuracy}")
        print(f"Fi score {f1}")

    except Exception as e:
        print(i, e)

In [ ]:
input_dim = len(relevant_features_combined)
fet = relevant_features_combined
for i in range(1,41):
    label_id = 1
    model_path = "<Path to the pretrained model>"
    X_test = stimulus_combined[fet]
    test_y = stimulus_combined['valence_category'].to_list()
    label1 = "Data of High and more valence_category"
    label2 = "Data of Low and less valence_category"
    label1 = lemmatize(label1)
    label1 = stop_words(label1)
    label1 = lowercase(label1)
    label1 = punctuations(label1)

    label2 = lemmatize(label2)
    label2 = stop_words(label2)
    label2 = lowercase(label2)
    label2 = punctuations(label2)
    try :
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        best_model = CLIPModel(mlp_input_dim = input_dim, mlp_output_dim = output_dim, device = device).to(device)
        best_model.load_state_dict(torch.load(model_path, map_location=device))
        pred = []
        text_embeddings  = best_model.text_encoder.text_tokens([label1])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label1 = text_embeddings.squeeze(1)
        encoded_label1 = encoded_label1.to(device)

        text_embeddings  = best_model.text_encoder.text_tokens([label2])
        text_embeddings = best_model.text_projection(text_embeddings)
        text_embeddings = torch.stack(text_embeddings).to(device)
        encoded_label2 = text_embeddings.squeeze(1)
        encoded_label2 = encoded_label2.to(device)
        X_test = np.array(X_test)
        for j in X_test:
            features = best_model.combined_encoder.get_hidden_embedding(torch.from_numpy(j).float().to(device))
            # print(features.shape)
            # print(encoded_label1.shape)
            # print(encoded_label2.shape)
            a = torch.matmul(features, encoded_label1.T) 
            b = torch.matmul(features, encoded_label2.T)
            # print(a)
            # print(b)
            value = 1 if a > b else 0
            pred.append(value)

        print(i, "valence")
        print(f"Prediction: {pred}")
        print(f"True Values: {test_y}")
        print(classification_report(test_y, pred))

        accuracy = accuracy_score(test_y, pred)
        f1 = f1_score(test_y, pred)

        print(f"Accuracy {accuracy}")
        print(f"Fi score {f1}")

    except Exception as e:
        print(i, e)

Code for MLP baseline model for comparison of the  result 

<as there is no baseline model given in the emognition paper>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, mean_squared_error, r2_score, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import random
import os
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import torchmetrics
from torchmetrics import F1Score
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer
from tqdm import tqdm
import neurokit2 as nk

warnings.filterwarnings('ignore')


In [69]:
def seed_everything(seed: int):
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed = 111 # (42, 43, 111)    
seed_everything(seed)

In [27]:
from sklearn.neural_network import MLPClassifier

def train_mlp(X_train, y_train_arousal, y_train_valence, X_test, y_test_arousal, y_test_valence, hidden_layer_sizes=(100,), random_seed=42):
    # Create and fit the MLP model for Arousal
    model_arousal_mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=random_seed)
    model_arousal_mlp.fit(X_train, y_train_arousal)  # Fit the model
    y_pred_arousal_mlp = model_arousal_mlp.predict(X_test)

    # Create and fit the MLP model for Valence
    model_valence_mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=random_seed)
    model_valence_mlp.fit(X_train, y_train_valence)  # Fit the model
    y_pred_valence_mlp = model_valence_mlp.predict(X_test)

    # Calculate accuracy for Arousal, Valence, and Task classification
    accuracy_arousal_mlp = accuracy_score(y_test_arousal, y_pred_arousal_mlp)
    balanced_acc_arousal_mlp = balanced_accuracy_score(y_test_arousal, y_pred_arousal_mlp)
    accuracy_valence_mlp = accuracy_score(y_test_valence, y_pred_valence_mlp)
    balanced_acc_valence_mlp = balanced_accuracy_score(y_test_valence, y_pred_valence_mlp)

    f1_arousal = f1_score(y_test_arousal, y_pred_arousal_mlp)
    f1_valence = f1_score(y_test_valence, y_pred_valence_mlp)

    r2_arousal = r2_score(y_test_arousal, y_pred_arousal_mlp)
    r2_valence = r2_score(y_test_valence, y_pred_valence_mlp)

    mse_arousal = mean_squared_error(y_test_arousal, y_pred_arousal_mlp)
    mse_valence = mean_squared_error(y_test_valence, y_pred_valence_mlp)

    return (accuracy_arousal_mlp, balanced_acc_arousal_mlp, accuracy_valence_mlp, balanced_acc_valence_mlp, 
            r2_arousal, r2_valence, mse_arousal, mse_valence, f1_arousal, f1_valence)


In [28]:
def validation(df, pi, relevant_features):

    accuracy_scores_a_mlp = {}
    balance_acc_list_a_mlp = {}
    accuracy_scores_v_mlp = {}
    balance_acc_list_v_mlp = {}
    r2_a_mlp = {}
    r2_v_mlp ={}
    mse_a_mlp = {}
    mse_v_mlp = {}
    f1_a_mlp = {}
    f1_v_mlp = {}
    accuracy_scores_s_mlp = {}
    balance_acc_list_s_mlp = {}
    r2_s_mlp = {}
    mse_s_mlp = {}
    f1_s_mlp = {}
    

    pi = list(set(pi))
    for i in pi:

        train_data = df[df['Participant ID'] != i]
        test_data = df[df['Participant ID'] == i]

        X_train = train_data[relevant_features]
        y_train_arousal = train_data['arousal_category'].astype(int)
        y_train_valence = train_data['valence_category'].astype(int)

        X_test = test_data[relevant_features]
        y_test_arousal = test_data['arousal_category'].astype(int)
        y_test_valence = test_data['valence_category'].astype(int)

        # Train the MLP classifier with specified hidden layers
        accuracy_arousal, balanced_acc_arousal, accuracy_valence, balanced_acc_valence, r2_a, r2_v, mse_a, mse_v, f1_a, f1_v, accuracy_task, balanced_acc_task, r2_task, mse_task, f1_task, num_zeros, num_ones = train_mlp(X_train, y_train_arousal, y_train_valence, X_test, y_test_arousal , y_test_valence)

        accuracy_scores_a_mlp[i] = accuracy_arousal*100
        balance_acc_list_a_mlp[i] = balanced_acc_arousal*100
        accuracy_scores_v_mlp[i] = accuracy_valence*100
        balance_acc_list_v_mlp[i] = balanced_acc_valence*100
        r2_a_mlp[i] = r2_a
        r2_v_mlp[i] = r2_v
        mse_a_mlp[i] = mse_a
        mse_v_mlp[i] = mse_v
        f1_a_mlp[i] = f1_a
        f1_v_mlp[i] = f1_v
        accuracy_scores_s_mlp[i] = accuracy_task*100
        balance_acc_list_s_mlp[i] = balanced_acc_task*100
        r2_s_mlp[i] = r2_task
        mse_s_mlp[i] = mse_task
        f1_s_mlp[i] = f1_task



    print(f"-----Average Accuracy Arousal-----")
    print(f"MLP: {sum(accuracy_scores_a_mlp.values()) / len(pi)}")

    print(f"-----Average Balanced Accuracy Arousal-----")
    print(f"MLP: {sum(balance_acc_list_a_mlp.values()) / len(pi)}")

    print(f"-----Average R2 Loss Arousal-----")
    print(f"MLP: {sum(r2_a_mlp.values()) / len(pi)}")

    print(f"-----Average MSE Loss Arousal-----")
    print(f"MLP: {sum(mse_a_mlp.values()) / len(pi)}")

    print(f"-----Average F1 Arousal-----")
    print(f"MLP: {sum(f1_a_mlp.values()) / len(pi)}")

    print(f"-----Average Accuracy Valence-----")
    print(f"MLP: {sum(accuracy_scores_v_mlp.values()) / len(pi)}")

    print(f"-----Average Balanced Accuracy Valence-----")
    print(f"MLP: {sum(balance_acc_list_v_mlp.values()) / len(pi)}")

    print(f"-----Average R2 Loss Valence-----")
    print(f"MLP: {sum(r2_v_mlp.values()) / len(pi)}")

    print(f"-----Average MSE Loss Valence-----")
    print(f"MLP: {sum(mse_v_mlp.values()) / len(pi)}")

    print(f"-----Average F1 Valence-----")
    print(f"MLP: {sum(f1_v_mlp.values()) / len(pi)}")

    print(f"-----Average Accuracy Stimulus-label-----")
    print(f"MLP: {sum(accuracy_scores_s_mlp.values()) / len(pi)}")

    print(f"-----Average Balanced Accuracy Stimulus-label-----")
    print(f"MLP: {sum(balance_acc_list_s_mlp.values()) / len(pi)}")

    print(f"-----Average R2 Loss Stimulus-label-----")
    print(f"MLP: {sum(r2_s_mlp.values()) / len(pi)}")

    print(f"-----Average MSE Loss Stimulus-label-----")
    print(f"MLP: {sum(mse_s_mlp.values()) / len(pi)}")

    print(f"-----Average F1 Stimulus-label-----")
    print(f"MLP: {sum(f1_s_mlp.values()) / len(pi)}")

In [24]:
relevant_features = ['ku_eda','sk_eda','dynrange','slope','variance','entropy','insc','fd_mean','max_scr','min_scr','nSCR','sumAmpSCR','sumRespSCR']
scaled_eda_fet = scaled_eda_fet[relevant_features + identifiers]
pi = scaled_eda_fet['Participant ID'].tolist()

In [ ]:
scaled_eda_fet.keys()

In [ ]:
columns_with_nan = scaled_eda_fet.columns[scaled_eda_fet.isna().any()].tolist()
columns_with_nan

In [ ]:
validation(stimulus_eda, pi, relevant_features)

In [ ]:
relevant_features = ['BPM','IBI','PPG_Rate_Mean','HRV_MedianNN','HRV_Prc20NN','HRV_MinNN','HRV_HTI','HRV_TINN','HRV_VHF','HRV_HFn','HRV_LnHF','HRV_SD1SD2','HRV_CVI','HRV_PSS','HRV_PAS','HRV_PI','HRV_C1d','HRV_C1a','HRV_DFA_alpha1','HRV_MFDFA_alpha1_Width','HRV_MFDFA_alpha1_Peak','HRV_MFDFA_alpha1_Mean','HRV_MFDFA_alpha1_Max','HRV_MFDFA_alpha1_Delta','HRV_MFDFA_alpha1_Asymmetry','HRV_ApEn','HRV_ShanEn','HRV_FuzzyEn','HRV_CD','HRV_HFD','HRV_KFD','HRV_LZC']
columns_with_nan = stimulus_ppg.columns[stimulus_ppg.isna().any()].tolist()
columns_with_nan
validation(stimulus_ppg, pi, relevant_features)

In [ ]:
relevant_features = ['ku_eda','sk_eda','dynrange','slope','variance','entropy','insc','fd_mean','max_scr','min_scr','nSCR','sumAmpSCR','sumRespSCR','BPM','IBI','PPG_Rate_Mean','HRV_MedianNN','HRV_Prc20NN','HRV_MinNN','HRV_HTI','HRV_TINN','HRV_VHF','HRV_HFn','HRV_LnHF','HRV_SD1SD2','HRV_CVI','HRV_PSS','HRV_PAS','HRV_PI','HRV_C1d','HRV_C1a','HRV_DFA_alpha1','HRV_MFDFA_alpha1_Width','HRV_MFDFA_alpha1_Peak','HRV_MFDFA_alpha1_Mean','HRV_MFDFA_alpha1_Max','HRV_MFDFA_alpha1_Delta','HRV_MFDFA_alpha1_Asymmetry','HRV_ApEn','HRV_ShanEn','HRV_FuzzyEn','HRV_CD','HRV_HFD','HRV_KFD','HRV_LZC']
columns_with_nan = pers_combined.columns[pers_combined.isna().any()].tolist()
columns_with_nan
validation(stimulus_combined, pi, relevant_features)